## Categorização de Tramitações de Projetos de Lei da Câmara dos Deputados
### Análise Exploratória dos dados de tramitação

**Autora**: Fernanda Scovino (FGV-EMAp)

### Objetivo
---
Criar uma classificação das tramitações de Projetos de Lei da Câmara dos Deputados a partir da análise de proximidade do seus textos de descrição.

**Base de dados:** [Dados de andamento dos Projetos de Lei de 1988 a 2018, disponíveis na API dos Dados Abertos da Câmara](http://www2.camara.leg.br/transparencia/dados-abertos/dados-abertos-legislativo/webservices/orgaos/obterandamento)

### Metodologia
---
A partir dos dados de descrição das tramitações, vamos utilizar pacotes como `nltk`, `gensim` e `tfidf` para verificar a frequência e a similaridade de palavras nos textos para identificarmos palavras chaves. Essas palavras chaves vão servir futuramente para categorizarmos as tramitações de acordo com os *clusters* gerados pelo algoritmo de *Topic Model*.

### Considerações
---

Na redução ao radical das palavaras foi utilizado o método *stemmer* do pacote `nltk`, que "corta" as palavras ao invés de reduzí-las a um lema, como é feito no método *lemmatizer*, pois este último não tem versão para português. Isso prejudica um pouco a leitura e o entendimento das palavras, que às vezes tem um corte muito drástico.

In [133]:
# DATA ANALYSIS
from copy import deepcopy
import pandas as pd

pd.options.display.max_columns = 999

import re
import nltk
import string
import gensim
import textract
import networkx as nx
from collections import Counter
from wordcloud import WordCloud

# VIZ TOOLS
import plotly
import plotly.graph_objs as go
import plotly.offline as offline

offline.init_notebook_mode(connected=True)

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)

colorscale = ['#dee0e0', '#b9bfbf', '#00D9FF', '#7c8282', '#0095D4', '#b5dde5', '#04b9d3', '#1C5D96'] # visual id

# FILES STORAGE
import os
cwd = os.getcwd()

RAW_DATA = cwd+'/raw_data/' # put primary data here
DATASET = cwd+'/datasets/' # put processed data here
GRAPH = cwd+'/graphs' 

Populating the interactive namespace from numpy and matplotlib


## IMPORTANTE!  NÃO RODAR AQUI!

### Como o arquivo 'trams_pls' é muito grande, não segui importar para o GitHub, mas os resultados da análise ainda estão aqui.

## Rode os dados a partir daqui: [Amostragem](#amostra)

### Importação dos dados

In [2]:
trams_pls = pd.read_csv(RAW_DATA+'trams_pls.csv')

In [3]:
trams_pls.columns

Index(['Unnamed: 0', 'index', 'idProposicao', 'ementa', 'situacao', 'numero',
       'ano', 'tipo', 'ultimaAcao_tramitacao_codOrgao',
       'ultimaAcao_tramitacao_orgao', 'ultimaAcao_tramitacao_descricao',
       'ultimaAcao_tramitacao_data', 'ultimaAcao_tramitacao_ordemDeTramitacao',
       'andamento_tramitacao_codOrgao', 'andamento_tramitacao_Orgao',
       'andamento_tramitacao_descricao', 'andamento_tramitacao_inteiroTeor',
       'andamento_tramitacao_codReuniao', 'andamento_tramitacao_data',
       'andamento_tramitacao_ordemDeTramitacao', 'data_captura'],
      dtype='object')

Abaixo podemos ver que as colunas `andamento_tramitacao_inteiroTeor` e `andamento_tramitacao_codReuniao` têm muitos dados faltantes, cerca de  60% e 90% dos valores nas colunas são nulos, respectivamente. 

Outras colunas, como `andamento_tramitacao_codOrgao`, `andamento_tramitacao_Orgao`,  `andamento_tramitacao_descricao`, `andamento_tramitacao_data` e `andamento_tramitacao_ordemDeTramitacao` também possuem valores nulos, mas que correpondem a somente 0,02% do total, aproximadamente.

In [4]:
trams_pls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428907 entries, 0 to 1428906
Data columns (total 21 columns):
Unnamed: 0                                 1428907 non-null int64
index                                      1428907 non-null int64
idProposicao                               1428907 non-null int64
ementa                                     1428907 non-null object
situacao                                   1428907 non-null object
numero                                     1428907 non-null int64
ano                                        1428907 non-null int64
tipo                                       1428907 non-null object
ultimaAcao_tramitacao_codOrgao             1428907 non-null int64
ultimaAcao_tramitacao_orgao                1428907 non-null object
ultimaAcao_tramitacao_descricao            1427607 non-null object
ultimaAcao_tramitacao_data                 1428907 non-null object
ultimaAcao_tramitacao_ordemDeTramitacao    1428907 non-null int64
andamento_tramitacao_co

In [5]:
trams_pls.head()

,Unnamed: 0,index,idProposicao,ementa,situacao,numero,ano,tipo,ultimaAcao_tramitacao_codOrgao,ultimaAcao_tramitacao_orgao,ultimaAcao_tramitacao_descricao,ultimaAcao_tramitacao_data,ultimaAcao_tramitacao_ordemDeTramitacao,andamento_tramitacao_codOrgao,andamento_tramitacao_Orgao,andamento_tramitacao_descricao,andamento_tramitacao_inteiroTeor,andamento_tramitacao_codReuniao,andamento_tramitacao_data,andamento_tramitacao_ordemDeTramitacao,data_captura
0,0,7791,575557,Cria Área de Livre Comércio no Município de Pa...,MESA - Arquivada,5494,2013,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,Arquivado nos termos do Artigo 105 do Regiment...,2015-01-31 00:00:00,61,2017.0,CINDRA,"Apresentação do Parecer do Relator, PRL 3 CIND...",http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2013-11-29 00:00:00,30.0,2018-03-22 19:20:47
1,1,7791,582462,Acrescenta alínea ao inciso II do art. 8º da L...,CPD - Aguardando Designação de Relator,5854,2013,PL,537480,CPD - Comissão de Defesa dos Direitos d...,"Recebimento pela CPD, com as proposições PL-34...",2018-02-26 00:00:00,97,4.0,MESA,Apense-se a este(a) o(a) PL-1915/2015.,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2015-06-23 00:00:00,33.0,2018-03-22 19:23:36
2,2,7791,584235,Altera a redação do § 3º do art. 71 da Consoli...,MESA - Arquivada,5909,2013,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,Arquivado nos termos do § 4° do art. 164 do RI...,2018-02-28 00:00:00,28,4.0,MESA,"Sujeito a arquivamento, nos termos do § 4º do ...",NaN,NaN,2018-02-07 00:00:00,26.0,2018-03-23 04:51:15
3,3,7791,589897,"Altera a Lei nº 12.305, de 2 de agosto de 2010...",MESA - Arquivada,6228,2013,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,"Arquivado, nos termos do art. 163 c/c o art.16...",2017-12-11 00:00:00,28,4.0,MESA,Apense-se à(ao) PL-5089/2013.Proposição Sujeit...,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,2013-09-09 00:00:00,11.0,2018-03-23 00:13:13
4,4,7791,598740,Altera dispositivo da Consolidação das Leis do...,CSSF - Tramitando em Conjunto,6659,2013,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,"Indeferido o Requerimento n. 3.629/2015, confo...",2016-01-12 00:00:00,32,2008.0,CDEIC,Recebimento pela CDEIC.,NaN,NaN,2013-11-13 00:00:00,17.0,2018-03-23 04:57:12


Pelo que vemos acima, as primeiras colunas da tabela contém informações permanentes de uma proposição como `idProposicao`, `ementa`, `numero`, `ano` e `tipo`, que caracterizam a mesma.

In [6]:
trams_pls.tipo.unique()

array(['PL'], dtype=object)

In [7]:
print(trams_pls.ano.min())
print(trams_pls.ano.max())

1988
2018


As colunas iniciadas com `ultimaAcao_tramitacao` referem-se à situação atual do projeto de lei, logo, são valores únicos para cada ṕrojeto.

Podemos verificar isso na contagem abaixo:

In [8]:
ultima_acao = trams_pls[['idProposicao', 
                         'ultimaAcao_tramitacao_codOrgao',
                         'ultimaAcao_tramitacao_orgao', 
                         'ultimaAcao_tramitacao_descricao',
                         'ultimaAcao_tramitacao_data', 
                         'ultimaAcao_tramitacao_ordemDeTramitacao']].drop_duplicates()

ultima_acao.groupby('idProposicao').count().sort_values('ultimaAcao_tramitacao_descricao', ascending=False)[:10]

,ultimaAcao_tramitacao_codOrgao,ultimaAcao_tramitacao_orgao,ultimaAcao_tramitacao_descricao,ultimaAcao_tramitacao_data,ultimaAcao_tramitacao_ordemDeTramitacao
idProposicao,,,,,
493149,2,2,2,2,2
1579163,2,2,2,2,2
490263,2,2,2,2,2
14919,1,1,1,1,1
474475,1,1,1,1,1
474140,1,1,1,1,1
474141,1,1,1,1,1
474175,1,1,1,1,1
474199,1,1,1,1,1


As colunas iniciadas com `andamento` carregam informações de cada tramitação dos projetos de lei, como o órgão e a data das tramitações. 

Em especial, a coluna `andamento_tramitacao_descricao` nos fornece o texto de descrição, como qual iremos trabalhar.

Existem 335 linhas com dados faltantes nessa coluna, que iremos desconsiderar por não ser uma quantidade representativa do total.

In [9]:
trams_pls[trams_pls.andamento_tramitacao_descricao.isnull()] # missing data

,Unnamed: 0,index,idProposicao,ementa,situacao,numero,ano,tipo,ultimaAcao_tramitacao_codOrgao,ultimaAcao_tramitacao_orgao,ultimaAcao_tramitacao_descricao,ultimaAcao_tramitacao_data,ultimaAcao_tramitacao_ordemDeTramitacao,andamento_tramitacao_codOrgao,andamento_tramitacao_Orgao,andamento_tramitacao_descricao,andamento_tramitacao_inteiroTeor,andamento_tramitacao_codReuniao,andamento_tramitacao_data,andamento_tramitacao_ordemDeTramitacao,data_captura
11463,11463,7980,18568,Dispõe sobre o salário mínimo e a manutenção d...,MESA - Arquivada,2743,2000,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,"Arquivado, nos termos do artigo 164, § 4º do R...",2006-03-10 00:00:00,23,2003.0,CCJC,NaN,NaN,NaN,2006-01-31 00:00:00,13.0,2018-03-23 03:57:45
14254,14254,8023,2092208,"Altera os artigos 155, 157, 180 e 266 do Decre...",CCJC - Tramitando em Conjunto,5853,2016,PL,2003,CCJC - Comissão de Constituição e Justiç...,"Recebimento pela CCJC, apensado ao PL-5845/2016",2016-08-15 00:00:00,19,186.0,CCP,NaN,NaN,NaN,2016-08-12 00:00:00,15.0,2018-03-22 18:17:45
14318,14318,8024,2092208,"Altera os artigos 155, 157, 180 e 266 do Decre...",CCJC - Tramitando em Conjunto,5853,2016,PL,2003,CCJC - Comissão de Constituição e Justiç...,"Recebimento pela CCJC, apensado ao PL-5845/2016",2016-08-15 00:00:00,19,186.0,CCP,NaN,NaN,NaN,2016-08-12 00:00:00,16.0,2018-03-22 18:17:45
17265,17265,8071,588807,Regulamenta as atividades de operador de Marke...,SERCO(SGM) - Aguardando Constituição de Comiss...,6170,2013,PL,180,PLEN - PLENÁRIO,"Ato da Presidência : Cria Comissão Especial, n...",2015-03-30 00:00:00,62,4.0,MESA,NaN,NaN,NaN,2014-11-25 00:00:00,41.0,2018-03-22 19:23:36
19590,19590,8109,2092207,Altera o artigo 184 da Lei Geral de Telecomuni...,CCJC - Tramitando em Conjunto,5852,2016,PL,2003,CCJC - Comissão de Constituição e Justiç...,"Recebimento pela CCJC, apensado ao PL-5846/2016",2017-07-13 00:00:00,18,186.0,CCP,NaN,NaN,NaN,2016-08-12 00:00:00,13.0,2018-03-22 18:17:45
19649,19649,8110,2092207,Altera o artigo 184 da Lei Geral de Telecomuni...,CCJC - Tramitando em Conjunto,5852,2016,PL,2003,CCJC - Comissão de Constituição e Justiç...,"Recebimento pela CCJC, apensado ao PL-5846/2016",2017-07-13 00:00:00,18,186.0,CCP,NaN,NaN,NaN,2016-08-12 00:00:00,14.0,2018-03-22 18:17:45
182934,182934,10748,354183,"Dispõe sobre a padronização, o registro, a ins...",PLEN - Pronta para Pauta,1254,2007,PL,186,CCP - COORDENAÇÃO DE COMISSÕES PERMANENTES,Encaminhada à publicação. Parecer da Comissão ...,2015-05-26 00:00:00,93,180.0,PLEN,NaN,NaN,NaN,2011-03-07 00:00:00,62.0,2018-03-22 23:04:34
187855,187855,10826,372158,Dispõe sobre a música e os eventos gospel.,Tranformada no(a) Lei Ordinária 12590/2012,2217,2007,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,Recebimento do Ofício nº 42/12(SF) encaminhand...,2012-02-08 00:00:00,76,4.0,MESA,NaN,NaN,NaN,2012-01-09 00:00:00,72.0,2018-03-22 23:19:46
187912,187912,10827,372158,Dispõe sobre a música e os eventos gospel.,Tranformada no(a) Lei Ordinária 12590/2012,2217,2007,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,Recebimento do Ofício nº 42/12(SF) encaminhand...,2012-02-08 00:00:00,76,4.0,MESA,NaN,NaN,NaN,2012-01-09 00:00:00,73.0,2018-03-22 23:19:46
187944,187944,10827,2023981,Dispõe sobre o visto temporário para o estrang...,MESA - Arquivada,3353,2015,PL,4,MESA - Mesa Diretora da Câmara dos Deput...,Encerramento automático do Prazo de Recurso 07...,2018-02-07 00:00:00,27,4.0,MESA,NaN,NaN,NaN,2017-12-13 00:00:00,25.0,2018-03-22 22:59:15


In [10]:
trams_pls_full = trams_pls[~trams_pls.andamento_tramitacao_descricao.isnull()][['andamento_tramitacao_descricao']]

In [11]:
trams_pls_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1428572 entries, 0 to 1428906
Data columns (total 1 columns):
andamento_tramitacao_descricao    1428572 non-null object
dtypes: object(1)
memory usage: 21.8+ MB


In [13]:
trams_pls_full.head()

,andamento_tramitacao_descricao
0,"Apresentação do Parecer do Relator, PRL 3 CIND..."
1,Apense-se a este(a) o(a) PL-1915/2015.
2,"Sujeito a arquivamento, nos termos do § 4º do ..."
3,Apense-se à(ao) PL-5089/2013.Proposição Sujeit...
4,Recebimento pela CDEIC.


<a id = 'amostra'></a>
### Amostragem

Selecionamos então uma amostra aleatória correspondente a 10% do total das tramitações para analisar os dados, cerca de 140 mil tramitações.

In [29]:
#trams_sample = trams_pls_full.sample(round(len(trams_pls_full)*0.1))

trams_sample = pd.read_csv('trams_sample.csv')

trams_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142857 entries, 238409 to 25999
Data columns (total 1 columns):
andamento_tramitacao_descricao    142857 non-null object
dtypes: object(1)
memory usage: 2.2+ MB


In [134]:
#trams_sample.to_csv(DATASET+'trams_sample.csv')

Dessa amostra, 60% será destinada para treino e 40% para teste.

In [34]:
#trams_train = trams_sample.sample(round(len(trams_sample)*0.6))
#trams_test = trams_sample.loc[trams_sample.index.difference(trams_train.index)]

trams_train = pd.read_csv(DATASET+'trams_train.csv')

trams_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85714 entries, 466267 to 728079
Data columns (total 1 columns):
andamento_tramitacao_descricao    85714 non-null object
dtypes: object(1)
memory usage: 1.3+ MB


In [143]:
trams_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57143 entries, 16 to 1428905
Data columns (total 1 columns):
andamento_tramitacao_descricao    57143 non-null object
dtypes: object(1)
memory usage: 892.9+ KB


In [144]:
#trams_train.to_csv(DATASET+'trams_train.csv')
#trams_test.to_csv(DATASET+'trams_test.csv')

### Limpeza e análise textual

In [145]:
list(trams_train.andamento_tramitacao_descricao)[:10]

['Recebido o Ofício n. 976/15, da CAPADR, comunicando que o PL n. 6.837/2013 recebeu pareceres divergentes nas comissões que lhe apreciaram o mérito.',
 'Recebimento pela CFT, com as proposições PL-8225/2014, PL-209/2015 apensadas.',
 'Arquivado nos termos do Artigo 105 do Regimento Interno. DCD 01 02 07 PAG 143 COL 01 SUPLEMENTO 01 AO Nº  21.',
 'Publicação inicial no DCD de 11/02/11 PÁG 5818 COL 02.',
 'Designado Relator, Dep. Mendes Ribeiro Filho (PMDB-RS)',
 'Abertura de Prazo para Emendas ao Projeto a partir de 14/05/2003',
 'Apresentação do Requerimento de Desapensação n. 4893/2016, pelo Deputado Rômulo Gouveia (PSD-PB), que: "Requer, nos termos regimentais, a desapensação do Projeto de Lei nº 4827/2016, apensado ao Projeto de Lei nº 7231/2014, com a finalidade de tramitarem em separados".',
 'Recebimento pela CSSF.',
 'Matéria não apreciada em face da não conclusão da apreciação da MPV 279/06, item 02 da pauta, com prazo encerrado.',
 'Retirado o destaque da Bancada do PT, para 

Agrupamos os textos para verificarmos detalhes sobre as palavras, como sua frequẽncia, "sinônimos", similaridade, etc.

In [146]:
trams_train['text'] = trams_train['andamento_tramitacao_descricao'] + ' '
texts = trams_train['text'].sum()

In [147]:
texts[0:1200]

'Recebido o Ofício n. 976/15, da CAPADR, comunicando que o PL n. 6.837/2013 recebeu pareceres divergentes nas comissões que lhe apreciaram o mérito. Recebimento pela CFT, com as proposições PL-8225/2014, PL-209/2015 apensadas. Arquivado nos termos do Artigo 105 do Regimento Interno. DCD 01 02 07 PAG 143 COL 01 SUPLEMENTO 01 AO Nº  21. Publicação inicial no DCD de 11/02/11 PÁG 5818 COL 02. Designado Relator, Dep. Mendes Ribeiro Filho (PMDB-RS) Abertura de Prazo para Emendas ao Projeto a partir de 14/05/2003 Apresentação do Requerimento de Desapensação n. 4893/2016, pelo Deputado Rômulo Gouveia (PSD-PB), que: "Requer, nos termos regimentais, a desapensação do Projeto de Lei nº 4827/2016, apensado ao Projeto de Lei nº 7231/2014, com a finalidade de tramitarem em separados". Recebimento pela CSSF. Matéria não apreciada em face da não conclusão da apreciação da MPV 279/06, item 02 da pauta, com prazo encerrado. Retirado o destaque da Bancada do PT, para votação em separado de expressão cons

Para analisarmos os dados textuais, primeiramente devemos limpar esses dados:
    
1. **Removendo pontuações e *stop words***, palavras consideradas irrelevantes para a classificação do texto, como conjunções, artigos. Essas palavras comumente tem alta frequência, pois aparecem em muitos textos, mas não agregam significado.

In [235]:
stop_pt = nltk.corpus.stopwords.words('portuguese')

list_words = texts.split()
list_words = [l.strip().lower() for l in list_words]
list_words = [l.strip(string.punctuation) for l in list_words] 
list_words = [l for l in list_words if l not in stop_pt]

freqdist = Counter(list_words)
freqdist.most_common(50)

[('dep', 20242),
 ('relator', 17557),
 ('parecer', 16794),
 ('projeto', 15921),
 ('apresentação', 15608),
 ('publicação', 14479),
 ('emendas', 14032),
 ('col', 13582),
 ('art', 12861),
 ('n', 11958),
 ('prazo', 11387),
 ('lei', 11173),
 ('dcd', 10747),
 ('01', 10450),
 ('pl', 9705),
 ('deputado', 9356),
 ('', 8612),
 ('ricd', 7873),
 ('termos', 7593),
 ('artigo', 7549),
 ('nº', 7184),
 ('pag', 7104),
 ('requerimento', 7103),
 ('pág', 6526),
 ('recebimento', 6178),
 ('proposição', 6123),
 ('despacho', 5732),
 ('02', 5636),
 ('encaminhada', 5587),
 ('inicial', 5425),
 ('24', 5278),
 ('comissão', 5210),
 ('comissões', 5122),
 ('sessões', 4913),
 ('designado', 4885),
 ('aprovação', 4675),
 ('apreciação', 4674),
 ('tramitação', 4622),
 ('apresentadas', 4545),
 ('encerrado', 4386),
 ('partir', 4358),
 ('54', 4336),
 ('ii', 4209),
 ('sujeita', 4093),
 ('apense-se', 4077),
 ('105', 3987),
 ('5', 3928),
 ('desarquivamento', 3898),
 ('substitutivo', 3891),
 ('regime', 3698)]

Dentre as 50 palavras mais frequentes, podemos ver que temos repetições por abreviações, como "deputado" e "dep", "artigo" e "art". 

**Também observamos alta ocorrência de valores numéricos, que vamos descartar.** Esses números são possivelmente proveninetes de datas, paginações, etc, que não nos interessam para a categorização dos textos.

In [236]:
list_words = [l for l in list_words if not any([c.isdigit() for c in l])]

freqdist = Counter(list_words)
freqdist.most_common(50)

[('dep', 20242),
 ('relator', 17557),
 ('parecer', 16794),
 ('projeto', 15921),
 ('apresentação', 15608),
 ('publicação', 14479),
 ('emendas', 14032),
 ('col', 13582),
 ('art', 12861),
 ('n', 11958),
 ('prazo', 11387),
 ('lei', 11173),
 ('dcd', 10747),
 ('pl', 9705),
 ('deputado', 9356),
 ('', 8612),
 ('ricd', 7873),
 ('termos', 7593),
 ('artigo', 7549),
 ('nº', 7184),
 ('pag', 7104),
 ('requerimento', 7103),
 ('pág', 6526),
 ('recebimento', 6178),
 ('proposição', 6123),
 ('despacho', 5732),
 ('encaminhada', 5587),
 ('inicial', 5425),
 ('comissão', 5210),
 ('comissões', 5122),
 ('sessões', 4913),
 ('designado', 4885),
 ('aprovação', 4675),
 ('apreciação', 4674),
 ('tramitação', 4622),
 ('apresentadas', 4545),
 ('encerrado', 4386),
 ('partir', 4358),
 ('ii', 4209),
 ('sujeita', 4093),
 ('apense-se', 4077),
 ('desarquivamento', 3898),
 ('substitutivo', 3891),
 ('regime', 3698),
 ('avulso', 3459),
 ('regimento', 3361),
 ('constituição', 3311),
 ('interno', 3310),
 ('deputados', 3053),
 ('

Devemos remover também acentuações para não repetimos palavras, como `pág` e `pag`.

In [237]:
import unidecode

list_words = [unidecode.unidecode(l) for l in list_words]
freqdist = Counter(list_words)
freqdist.most_common(50)

[('dep', 20242),
 ('relator', 17557),
 ('parecer', 16794),
 ('projeto', 15921),
 ('apresentacao', 15610),
 ('publicacao', 14481),
 ('emendas', 14032),
 ('pag', 13630),
 ('col', 13582),
 ('art', 12861),
 ('n', 11958),
 ('prazo', 11387),
 ('lei', 11173),
 ('dcd', 10747),
 ('pl', 9705),
 ('deputado', 9356),
 ('', 8612),
 ('ricd', 7873),
 ('termos', 7593),
 ('artigo', 7549),
 ('no', 7184),
 ('requerimento', 7103),
 ('recebimento', 6178),
 ('proposicao', 6124),
 ('despacho', 5732),
 ('encaminhada', 5587),
 ('inicial', 5425),
 ('comissao', 5210),
 ('comissoes', 5122),
 ('sessoes', 4914),
 ('designado', 4885),
 ('aprovacao', 4676),
 ('apreciacao', 4674),
 ('tramitacao', 4622),
 ('apresentadas', 4545),
 ('encerrado', 4386),
 ('partir', 4358),
 ('ii', 4209),
 ('sujeita', 4093),
 ('apense-se', 4077),
 ('desarquivamento', 3898),
 ('substitutivo', 3891),
 ('regime', 3698),
 ('avulso', 3459),
 ('regimento', 3361),
 ('constituicao', 3312),
 ('interno', 3310),
 ('deputados', 3053),
 ('justica', 3050)

In [238]:
labels, values = zip(*freqdist.most_common(50))

trace = go.Bar(x=labels, y=values)
layout = go.Layout(title='Frequência de palavras nos textos de tramitação', 
                  yaxis=dict(title='Número de ocorrências'), xaxis=dict(tickangle=55))

fig = go.Figure(data=[trace], layout=layout)
offline.iplot(fig)

Outros detalhe é que temos repetições de palavras através de flexões nominais e verbais, que carregam o mesmo significado. Temos um exemplo abaixo:

In [239]:
print(freqdist['encaminhada'])
print(freqdist['encaminhado'])
print(freqdist['encaminhamento'])

5587
1246
800


Logo, para evitar esse tipo de repetição, vamos reduzir as palavras aos seu radicais!

In [240]:
list_words = list(filter(lambda x: x != '', list_words)) # removing empty str

In [241]:
stemmer = nltk.stem.RSLPStemmer()

list_words2 = [stemmer.stem(l) for l in list_words]

freqdist2 = Counter(list_words2)
freqdist2.most_common(50)

[('dep', 20251),
 ('rela', 19205),
 ('parec', 17114),
 ('projet', 16732),
 ('emend', 16167),
 ('apresentaca', 15610),
 ('publicaca', 14481),
 ('pag', 13744),
 ('col', 13602),
 ('deput', 13524),
 ('art', 13008),
 ('n', 11958),
 ('lei', 11482),
 ('praz', 11421),
 ('dcd', 10898),
 ('pl', 9865),
 ('receb', 8740),
 ('encaminh', 8168),
 ('ricd', 7873),
 ('no', 7661),
 ('artig', 7650),
 ('term', 7605),
 ('requer', 7225),
 ('proposica', 6124),
 ('apresent', 5816),
 ('despach', 5755),
 ('encerr', 5650),
 ('design', 5555),
 ('inic', 5442),
 ('desarquiv', 5422),
 ('ordin', 5388),
 ('comissa', 5210),
 ('comisso', 5122),
 ('sesso', 4914),
 ('sujeit', 4815),
 ('aprovaca', 4676),
 ('apreciaca', 4674),
 ('tramitaca', 4622),
 ('part', 4478),
 ('ii', 4211),
 ('apense-s', 4077),
 ('substitu', 4050),
 ('regim', 3702),
 ('dest', 3684),
 ('avuls', 3463),
 ('reg', 3454),
 ('arquiv', 3437),
 ('intern', 3315),
 ('constituica', 3312),
 ('apens', 3286)]

**O problema dessa redução ao radical é que perdemos o significado de muitas palavras!** 

Por exemplo, olhando para o 2º termo mais frequente, "rela", não sabemos exatamente quais palavras podem ter esse radical, pois é algo muito genérico. Por exemĺo, "relação", "relatoria", e outras palavras que têm diferentes significados podem ter sido reduzidas a esse radical.

In [242]:
labels, values = zip(*freqdist2.most_common(50))

trace = go.Bar(x=labels, y=values)
layout = go.Layout(title='Frequência dos radicais nos textos de tramitação', 
                  yaxis=dict(title='Número de ocorrências'), xaxis=dict(tickangle=55))

fig = go.Figure(data=[trace], layout=layout)
offline.iplot(fig)

Uma saída é fazermos a lematização das palavras utilizando o pacote `textblob`

In [229]:
from textblob import Word

list_words3 = [Word(l).lemmatize() for l in list_words]

freqdist3 = Counter(list_words3)
freqdist3.most_common(50)

ImportError: No module named 'textblob'

In [ ]:
labels, values = zip(*freqdist3.most_common(50))

trace = go.Bar(x=labels, y=values)
layout = go.Layout(title='Frequência dos lemas nos textos de tramitação', 
                  yaxis=dict(title='Número de ocorrências'), xaxis=dict(tickangle=55))

fig = go.Figure(data=[trace], layout=layout)
offline.iplot(fig)